In [235]:
'''Numerical Study'''

import matplotlib.pyplot as plt

from matplotlib.ticker import MultipleLocator, FormatStrFormatter

import numpy as np

import pandas as pd

import folium

import geopandas as gp

import copy

from shapely.geometry import Point, Polygon

In [236]:
Strategy='Local'

Geometry=np.load('./Data/Initialization/Geo.npy',allow_pickle=True).item()

'''Load data'''

Order_df=pd.read_csv('./Data/Processed/Dispatched_Order_'+str(Strategy)+'.csv')

Order_df=Order_df.drop(columns=['Unnamed: 0'])

Driver_df=pd.read_csv('./Data/Processed/Dispatched_Driver_'+str(Strategy)+'.csv')

Driver_df=Driver_df.drop(columns=['Unnamed: 0'])

Broadcast_df=pd.read_csv('./Data/Processed/Broadcast_'+str(Strategy)+'.csv')

Broadcast_df=Broadcast_df.drop(columns=['Unnamed: 0'])

# Quatitive_results_5=np.load('./Data/Processed/Quatitive_results_'+str(5)+'.npy',allow_pickle=True)

# Quatitive_results_10=np.load('./Data/Processed/Quatitive_results_'+str(10)+'.npy',allow_pickle=True)

# Example

In [237]:
step=3559

EXAM_Broadcast=copy.copy(Broadcast_df[Broadcast_df['step']==step])

EXAM_Order_df=copy.copy(Order_df[(Order_df['response_step']==step)])

EXAM_Driver_df=copy.copy(Driver_df[(Driver_df['step']==step)])


EXAM_Driver_df=EXAM_Driver_df.astype({'driver_id': 'int32'})

EXAM_Driver_df=EXAM_Driver_df.astype({'driver_id': 'str'})

EXAM_Order_df=EXAM_Order_df.merge(EXAM_Driver_df[['driver_id','longitude','latitude']],on='driver_id',how='left')



# Broadcasting 

In [238]:


'''Background'''

background = folium.Map([22.648883, 114.045011],tiles='CartoDB positron',zoom_start=10)

'''Zones'''

folium.Choropleth(
    geo_data=Geometry['Shenzhen'],
    fill_color='blue',
    fill_opacity=0.0,
    name='Zone').add_to(background)

folium.LayerControl().add_to(background)

for idx,row in EXAM_Broadcast.iterrows():
    
    origin=[float(x) for x in row['origin'].replace('POINT','').replace('(','').replace(')','').strip(' ').split(' ')][::-1];
    loc=[float(x) for x in row['loc'].replace('POINT','').replace('(','').replace(')','').strip(' ').split(' ')][::-1]
    
    folium.Circle(
    radius=10,
    location=origin,
    color='red',
    fill=True,
    ).add_to(background)
    
    folium.Circle(
    radius=10,
    location=loc,
    color='blue',
    fill=True,
    ).add_to(background)
    
    if row['response']==0:
    
        folium.PolyLine(
        [origin,loc],
        color='black',
        weight=1, 
        opacity=0.5
        ).add_to(background)
        
    else:
        
        folium.PolyLine(
        [origin,loc],
        color='green',
        weight=1.5, 
        opacity=0.8
        ).add_to(background)
        
for idx,row in EXAM_Order_df[EXAM_Order_df['driver_id']!='Canceled'].iterrows():
    
    origin=[row['start_lat'],row['start_lng']]
    
    loc=[row['latitude'],row['longitude']]
    
    folium.PolyLine(
        [origin,loc],
        color='red',
        weight=2, 
        opacity=1
        ).add_to(background)
    
        

background.save('Broadcast_example_'+Strategy+'.html')

# Evaluation

In [239]:
'''Served orders'''

print(Order_df.loc[(Order_df['driver_id']!='Canceled')&(Order_df['driver_id']!='Waiting')].shape[0])

print(Order_df.loc[(Order_df['driver_id']!='Canceled')&(Order_df['driver_id']!='Waiting')].shape[0]/11424)

print(Order_df.loc[(Order_df['driver_id']=='Canceled')].shape[0])

print(Order_df.loc[(Order_df['driver_id']=='Canceled')].shape[0]/11424)

print(Order_df.loc[(Order_df['driver_id']=='Waiting')].shape[0])

print(Order_df.loc[(Order_df['driver_id']=='Waiting')].shape[0]/11424)

print('GMV: ',Order_df.loc[(Order_df['driver_id']!='Canceled')&(Order_df['driver_id']!='Waiting'),'fee'].sum())

10704
0.9369747899159664
549
0.04805672268907563
171
0.014968487394957984
GMV:  1255880


In [240]:
'''Average Response time'''

Order_Stat=Order_df.loc[(Order_df['create_step']<=3600)&(Order_df['driver_id']!='Canceled')&(Order_df['driver_id']!='Waiting')]

Order_Stat['Response_time']=Order_Stat.apply(lambda x:(x['response_step']-x['create_step'])*10,axis=1)

Order_Stat=Order_Stat[['order_id','pickup_step','Response_time']]

Order_Stat['Response_time'].mean()


/home/data/marco.yang/.conda/envs/marco/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


11.145366218236173

In [241]:
'''Average Pick-up time'''

Order_Stat=Order_df.loc[(Order_df['create_step']<=3600)&(Order_df['driver_id']!='Canceled')&(Order_df['driver_id']!='Waiting')]

Order_Stat['Pickup_time']=Order_Stat.apply(lambda x:(x['pickup_step']-x['response_step'])*10,axis=1)

Order_Stat=Order_Stat[['order_id','pickup_step','Pickup_time']]

Order_Stat['Pickup_time'].mean()


/home/data/marco.yang/.conda/envs/marco/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


207.2552316890882

In [242]:
def Get_Travel_time(start_lng,start_lat,end_lng,end_lat,pickup_step):
    pnt1=Point(start_lat,start_lng);
    pnt2=Point(end_lat,end_lng);
    dis=pnt1.distance(pnt2)*111*1.3
    travel_time=int(dis/0.1)
    
    if pickup_step+travel_time>=3600:
        
        return 3600-pickup_step
    
    else:
        return travel_time
    


'''Average Occupied rate'''

Order_Stat=Order_df.loc[(Order_df['create_step']<=3600)&(Order_df['pickup_step']<=3600)&(Order_df['driver_id']!='Canceled')&(Order_df['driver_id']!='Waiting')]


Order_Stat['duration']=Order_Stat.apply(lambda x:Get_Travel_time(x['start_lng'],\
                                                                x['start_lat'],\
                                                                x['end_lng'],\
                                                                x['end_lat'],\
                                                                x['pickup_step']),axis=1)




Order_Stat['duration'].sum()/(4241*1080)

/home/data/marco.yang/.conda/envs/marco/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.3380341376509733